# United Nations General Assembly Resolutions
## Network Analysis and Alliance Structures

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

dirname=os.path.dirname

ROOT_DIR = os.path.join('..', '..')
FINAL_DATA_PATH = os.path.join(ROOT_DIR, 'data', 'final')
ERROR_LOG = os.path.join(ROOT_DIR, 'error-logs')
OUTPUT_PATH = os.path.join(ROOT_DIR, 'output', 'network-analysis')
TOPIC_PATH = os.path.join(ROOT_DIR, 'output', 'topic-modeling', 'document_topics.csv')
CLUSTER_PATH = os.path.join(ROOT_DIR, 'output', 'unsupervised', 'clusters.csv')

plt.style.use("seaborn")
plt.rcParams['figure.dpi'] = 75
plt.rcParams['savefig.dpi'] = 300

### Load Data

In [2]:
era_names = ['Early Cold War (1946-1960)', 'Transitional Period (1971-1990)', 'Post-Cold War (1991-2022)']


def get_era(year): 
    if year <= 1960:
        return era_names[0]
    if year <= 1990: 
        return era_names[1]
    return era_names[2]

def transform_votes(df): 
    vote_dict = {
        'Y': 1,
        'N': -1,
        'A': 0,
        'X': 0
    }
    
    new_votes = df.dropna(axis=1, thresh=df.shape[0] * .5)
    for country in new_votes.columns: 
        new_votes.loc[:, country] = new_votes.loc[:, country].map(vote_dict)
    new_votes.fillna(0, inplace=True)
    return new_votes


votes = pd.read_json(os.path.join(FINAL_DATA_PATH, 'votes.json'))
topics = pd.read_csv(TOPIC_PATH)
topics = topics[topics['Resolution'].isin(votes.index)].reset_index(drop=True)
votes = votes[votes.index.isin(topics['Resolution'])]

md = pd.read_json(os.path.join(FINAL_DATA_PATH, 'metadata.json'))
md['Vote date'] = pd.to_datetime(md['Vote date'])
md['Year'] = pd.DatetimeIndex(md['Vote date']).year.astype(int)
md['era'] = md.pop('Year').apply(get_era)

votes = votes.merge(md[['Resolution', 'era']], how='inner', left_index=True, right_on='Resolution').set_index(['era', 'Resolution']).sort_index()
topics = topics.merge(md[['Resolution', 'era']], how='inner', on='Resolution').set_index(['era', 'Resolution']).round(4).sort_index()

clusters = pd.read_csv(CLUSTER_PATH)

### Early Cold War (1946-1960)

#### Building the Adjacency Matrices

In [223]:
def build_matrices(votes, topics): 
    matrices = [np.zeros((votes.shape[1], votes.shape[1])) for _ in range(topics.shape[1])]
    for i1, c1 in enumerate(votes.columns): 
        for i2, c2 in enumerate(votes.columns): 
            if c1 == c2:
                continue
            raw_agreement = votes[c1] * votes[c2]
            agreements = raw_agreement @ topics
            for topic, a in enumerate(agreements):
                matrices[topic][i1, i2] = a
    return matrices

def build_graphs(votes, topics): 
    graphs = []
    matrices = build_matrices(votes, topics)
    for matrix in matrices: 
        g = nx.convert_matrix.from_numpy_matrix(matrix)
        g = nx.relabel_nodes(g, {i : w for i, w in enumerate(votes.columns)})
        graphs.append(g)
    return graphs 

def normalize(weights):
    weights = np.array(weights)
    widths = (weights - weights.min()) / (weights.max() - weights.min()) / 5
    return widths

def draw_graphs(graphs, votes, era, names=True, save=True, show=False):

    plt.ioff()
    if show: 
        plt.ion()
    colors = clusters[clusters['era']==era].set_index('country').loc[votes.columns, :]['color']
    k = round(len(votes.columns) / 50
)
    for topic, gr in enumerate(graphs):
        if topic != 0: 
            continue

        g = gr.to_directed()
        layout = nx.spring_layout(g, weight='weight', iterations= 50, seed=2, k=k)

        fig, ax = plt.subplots(figsize = (12,12))
        
        if names: 
            folder = 'named'
            labels = {n:n for i, n in enumerate(g.nodes())}
        else:
            folder = 'unnamed'
            labels = None

        widths = normalize([d['weight'] for n1, n2, d in g.edges(data=True)])
        edgelist = np.array([(n1, n2) for n1, n2, d in g.edges(data=True)])[widths > .1]
        final_edgelist = []
        for n1, n2 in edgelist:
            if (n2, n1) not in final_edgelist:
                final_edgelist.append((n1, n2))

        nx.draw(g, ax = ax, pos = layout, labels = labels,
                width=widths,
                edgelist=final_edgelist,
                alpha=.9, 
                node_size=120,
                node_color=colors,
                edge_color='grey',
                connectionstyle="arc3,rad=0.2", arrowstyle='-', font_size=10)
        
        if save: 
            fig_path = os.path.join(OUTPUT_PATH, folder, f"era-{era}", f'graph-{topic}.png')
            fig.savefig(fig_path)
        plt.close(fig)

In [165]:
ecw_votes = transform_votes(votes.loc[era_names[0]])
ecw_topics = topics.loc[era_names[0]]

ecw_graphs = build_graphs(ecw_votes, ecw_topics)

In [224]:
# draw_graphs(ecw_graphs, ecw_votes, era=0)
draw_graphs(ecw_graphs, ecw_votes, era=0, names=False)

### Transitional Period (1971-1990)

#### Building the Adjacency Matrices

In [93]:
tp_votes = transform_votes(votes.loc[era_names[1]])
tp_topics = topics.loc[era_names[1]]

tp_graphs = build_graphs(tp_votes, tp_topics)

In [221]:
draw_graphs(tp_graphs, tp_votes, era=1)
draw_graphs(tp_graphs, tp_votes, era=1, names=False)

### Post-Cold War (1991-2022)

In [95]:
pcw_votes = transform_votes(votes.loc[era_names[2]])
pcw_topics = topics.loc[era_names[2]]

pcw_graphs = build_graphs(pcw_votes, pcw_topics)

In [106]:
draw_graphs(tp_graphs, tp_votes, era=1)
draw_graphs(tp_graphs, tp_votes, era=1, names=False)

In [134]:
len(pcw_graphs[0].nodes())

192